# 🦜🔗 LangChain agent using a db

In [ ]:
%pip install -r requirements.txt
! [ ! -f northwind.db ] && bunzip2 northwind.db.bz2

Wir brauchen eine kleine Test-Datenbank: https://database.guide/2-sample-databases-sqlite/
Die Kurzanleitung ist wie folgt:

- Sqlite3 installieren
- Dieses [File](https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql) als Chinook_Sqlite.sql abspeichern
- Run sqlite3 Chinook.db (öffnet ein interaktives sqlite Terminal)
- Run .read Chinook_Sqlite.sql
- Test SELECT * FROM Artist LIMIT 10;

In [41]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
from langgraph.prebuilt import create_agent_executor
from langchain.agents import create_openai_functions_agent
from langchain import hub
from helpers import llm
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import (
    ChatPromptTemplate
)
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
my_llm = llm._prepare()[0]._prepare()[0] #HACK HACK HACK #TODO REWORK
toolkit = SQLDatabaseToolkit(db=db, llm=my_llm)
tools = toolkit.get_tools()
prompt: ChatPromptTemplate = hub.pull("reactagent/sql")
prompt = prompt.partial(dialect=toolkit.dialect, top_k=10)
agent_runnable = create_openai_functions_agent(llm, tools, prompt)
app = create_agent_executor(agent_runnable, tools)

In [ ]:
from typing import List, Tuple
from langchain_core.agents import AgentActionMessageLog, AgentFinish
inputs = {"input": "List the total sales per country. Which country's customers spent the most?"}
# Dies hier ist kein "echtes" Agenten-Streaming. Die gestreamten Chunks sind die kompletten Zwischenschritte des Agenten und der Tools, keine Token.
for chunk in app.stream(inputs):
    for key, value in chunk.items():
        if key == "agent":
            outcome  = value.get("agent_outcome")
            if isinstance(outcome, AgentActionMessageLog):
                print(f"Agent log:\n\n{outcome.log.strip()}")
            elif isinstance(outcome, AgentFinish):
                print(f"Agent finished:\n\n{outcome.log.strip()}")
            print("\n----------------------------------------------------------------------------------------\n\n")
        elif key == "action":
            steps: List[Tuple[AgentActionMessageLog, str]] = value.get("intermediate_steps")
            for index, step in enumerate(steps):
                print(f"Tool log:\n\n{step[1].strip()}")
                if index < len(steps) - 1:
                    print("----------------")
            print("\n----------------------------------------------------------------------------------------\n\n")
        elif key == "__end__":
            print("Done")

Die alten Befehle. Was tun die?  War die alte DB besser?

- db = SQLDatabase.from_uri("sqlite:///northwind.db")
- agent_executor.run("Where do i find the orders?")
- agent_executor.run("Are there any order from Munich?")
- agent_executor.run("Which Employee has the most orders?")
